## Create Physicochemical Properties

In [2]:
import os
import pandas as pd
import sys
!"{sys.executable}" -m pip install tidytcells

In [3]:
# set precision of mhc and V/J values (gene or allele)
precision = 'gene'

In [4]:
# this function is not thread safe
def create_folders_if_not_exists(folders):
  for path in folders:
    if not os.path.exists(path):
      os.makedirs(path)

In [5]:
pipeline_data = '../../data'
pipeline_data_plain = f'{pipeline_data}/plain_datasets'
pipeline_data_cleaned = f'{pipeline_data}/cleaned_datasets'
pipeline_data_concatenated = f'{pipeline_data}/concatenated_datasets'
pipeline_data_splitted = f'{pipeline_data}/splitted_datasets'
pipeline_data_temp_bucket = f'{pipeline_data}/temp'

pipeline_folders = [pipeline_data, pipeline_data_plain, pipeline_data_cleaned, pipeline_data_concatenated, pipeline_data_splitted, pipeline_data_temp_bucket]

create_folders_if_not_exists(pipeline_folders)

## PLE

In [24]:
import importlib.util


ple_path = "../../data_scripts/ple_embedding.py"

# Lade das Modul manuell
spec = importlib.util.spec_from_file_location("ple_embedding", ple_path)
ple_embedding = importlib.util.module_from_spec(spec)
spec.loader.exec_module(ple_embedding)

# Jetzt kannst du Funktionen direkt verwenden
encode_dataframe = ple_embedding.encode_dataframe
ple_encode_sequence = ple_embedding.ple_encode_sequence
print("PLE-Embedding-Modul erfolgreich geladen!")


PLE-Embedding-Modul erfolgreich geladen!


In [14]:
import pandas as pd
import numpy as np

# Lade die Daten
df_paired = pd.read_csv("../../data/splitted_datasets/gene/paired/train.tsv", sep="\t")

# Wende PLE auf TCR-Sequenzen an
ple_TRA = encode_dataframe(df_paired, "TRA_CDR3")
ple_TRB = encode_dataframe(df_paired, "TRB_CDR3")
ple_Epi = encode_dataframe(df_paired, "Epitope")

def pad_sequences(sequences, max_len=None):
    """
    Füllt alle Sequenzen mit Nullen auf die gleiche Länge auf.
    :param sequences: Liste von NumPy-Arrays mit PLE-Embeddings
    :param max_len: Maximale Länge der Sequenzen (falls None, wird längste Sequenz verwendet)
    :return: 3D-NumPy-Array (Anzahl der Sequenzen, max_len, Feature-Dimension)
    """
    if max_len is None:
        max_len = max(seq.shape[0] for seq in sequences)  # Finde die längste Sequenz

    feature_dim = sequences[0].shape[1]  # Anzahl der physikochemischen Eigenschaften
    padded_seqs = np.zeros((len(sequences), max_len, feature_dim))  # (num_samples, max_len, feature_dim)

    for i, seq in enumerate(sequences):
        padded_seqs[i, :seq.shape[0], :] = seq  # Kopiere Originalwerte

    return padded_seqs

# Padding anwenden, um gleiche Länge für alle Sequenzen zu haben
ple_TRA_padded = pad_sequences(ple_TRA)
ple_TRB_padded = pad_sequences(ple_TRB)
ple_Epi_padded = pad_sequences(ple_Epi)

# Speichern der PLE-Embeddings mit Padding
np.savez_compressed("../../data/embeddings/paired/gene/TRA_PLE_padded.npz", embeddings=ple_TRA_padded)
np.savez_compressed("../../data/embeddings/paired/gene/TRB_PLE_padded.npz", embeddings=ple_TRB_padded)
np.savez_compressed("../../data/embeddings/paired/gene/Epitope_PLE_padded.npz", embeddings=ple_Epi_padded)

print("PLE-Embeddings erfolgreich mit Padding erstellt und gespeichert!")

PLE-Embeddings erfolgreich mit Padding erstellt und gespeichert!


In [16]:
df_beta = pd.read_csv("../../data/splitted_datasets/gene/beta/train.tsv", sep="\t")

# Wende PLE auf die Beta-Sequenzen an
ple_TRB = encode_dataframe(df_beta, "TRB_CDR3")
ple_Epi = encode_dataframe(df_beta, "Epitope")

# Padding anwenden, um gleiche Länge für alle Sequenzen zu haben
ple_TRB_padded = pad_sequences(ple_TRB)
ple_Epi_padded = pad_sequences(ple_Epi)

# Speichern der PLE-Embeddings mit Padding
np.savez_compressed("../../data/embeddings/beta/gene/TRB_PLE_padded.npz", embeddings=ple_TRB_padded)
np.savez_compressed("../../data/embeddings/beta/gene/Epitope_PLE_padded.npz", embeddings=ple_Epi_padded)

print("PLE-Embeddings für Beta erfolgreich mit Padding erstellt und gespeichert!")

/tmp/ipykernel_2414/325641139.py:1: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_beta = pd.read_csv("../../data/splitted_datasets/gene/beta/train.tsv", sep="\t")


PLE-Embeddings für Beta erfolgreich mit Padding erstellt und gespeichert!


In [27]:
test_seq = "AVFDRKSDAK"  # Beispiel einer Epitope-Sequenz
expected_ple = ple_encode_sequence(test_seq)

print("\nErwartete PLE-Werte für:", test_seq)
for aa, values in zip(test_seq, expected_ple):
    print(f"{aa}: {values}")
#Alle Werte stimmen mit der physikochemischen Tabelle überein!


Erwartete PLE-Werte für: AVFDRKSDAK
A: [1.8  0.62 0.05]
V: [4.2  1.08 0.36]
F: [2.8  1.19 0.42]
D: [-3.5  -0.9  -0.49]
R: [-4.5  -2.53 -0.32]
K: [-3.9  -1.5  -0.34]
S: [-0.8  -0.18 -0.05]
D: [-3.5  -0.9  -0.49]
A: [1.8  0.62 0.05]
K: [-3.9  -1.5  -0.34]


In [28]:
epi_ple = np.load("../../data/embeddings/paired/gene/Epitope_PLE_padded.npz")["embeddings"]

print("Vergleich gespeicherte vs. berechnete PLE-Werte:")
print(epi_ple[0])  # Erste gespeicherte Epitope-Kodierung
print(expected_ple)  # Neu berechnete Kodierung

Vergleich gespeicherte vs. berechnete PLE-Werte:
[[ 1.8   0.62  0.05]
 [ 4.2   1.08  0.36]
 [ 2.8   1.19  0.42]
 [-3.5  -0.9  -0.49]
 [-4.5  -2.53 -0.32]
 [-3.9  -1.5  -0.34]
 [-0.8  -0.18 -0.05]
 [-3.5  -0.9  -0.49]
 [ 1.8   0.62  0.05]
 [-3.9  -1.5  -0.34]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]]
[[ 1.8   0.62  0.05]
 [ 4.2   1.08  0.36]
 [ 2.8   1.19  0.42]
 [-3.5  -0.9  -0.49]
 [-4.5  -2.53 -0.32]
 [-3.9  -1.5  -0.34]
 [-0.8  -0.18 -0.05]
 [-3.5  -0.9  -0.49]
 [ 1.8   0.62  0.05]
 [-3.9  -1.5  -0.34]]


## Periodische Embeddings